# Shot detection

In this notebook, we shall take a set of IIIF video Manifests, and using the [distant viewing toolkit](https://github.com/distant-viewing/dvt/tree/main) to detect different shots. Then we shall take these results and modify the initial Manifest accordingly and save it as a new Manifest with time-based annotations for each shot.

## 1. Setup

First let's install and import all of the libraries we shall need.

In [ ]:
!pip install -q git+https://github.com/jdchart/millefeuille-api.git
!pip install -q git+https://github.com/distant-viewing/dvt.git
!mkdir -p /root/.cache/torch/hub/checkpoints/

import mfapi
import dvt
import os
import requests
import json
import shutil
from google.colab import files

## 2. Get sources

Next, we shall collect the IIIF Manifests we wish to analyse.

In [ ]:
manifest_list = [
    "https://files.tetras-libre.fr/manifests/plozevet/sources/2e5e742f-2ce2-4ea2-9040-de0af66758ef.json",
    "https://files.tetras-libre.fr/manifests/plozevet/sources/46bc2cbe-57c7-48c9-a7c1-ecb914928d83.json",
    "https://files.tetras-libre.fr/manifests/plozevet/sources/a9317307-7518-49ff-b5b0-7b542593b35f.json",
    "https://files.tetras-libre.fr/manifests/plozevet/sources/d77eb578-8006-48d7-9854-df5b092fd8f9.json",
    "https://files.tetras-libre.fr/manifests/plozevet/sources/d19670f7-81c9-4272-8293-6fa4b45536bf.json"
]

## 3. Analysis

Now we shall use the distant viewing toolkit's `AnnoShotBreaks` class to detect shots in the source video.

In [ ]:
# Make a folder to output results:
if os.path.isdir("analysis_results") == False:
  os.makedirs("analysis_results")

# The shot detection model:
anno_breaks = dvt.AnnoShotBreaks()

# Some utility functions:
def dl_video(url, out_path):
  if os.path.isfile(out_path) == False:
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(out_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)

def parse_results(analysis_result, frame_rate):
  ret = {"shots" : []}
  for i in range(len(analysis_result["scenes"]["start"])):
        ret["shots"].append([analysis_result["scenes"]["start"][i] / frame_rate, analysis_result["scenes"]["end"][i] / frame_rate])
  return ret

def write_json(data, path):
  with open(path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii = False, indent = 2)
  files.download(path)

# Treat each video:
for manifest_path in manifest_list:
  print(f"Treating {manifest_path}...")

  media_url = mfapi.read_manifest(manifest_path).get_media().id

  dl_video(media_url, os.path.basename(media_url))
  frame_rate = dvt.video_info(os.path.basename(media_url))["fps"]
  out_breaks = anno_breaks.run(os.path.basename(media_url))

  result_parse = parse_results(out_breaks, frame_rate)
  result_parse["manifest_path"] = manifest_path
  result_parse["media_path"] = media_url

  write_json(result_parse, os.path.join("analysis_results", os.path.splitext(os.path.basename(media_url))[0] + "_shot_detection.json"))

This process can be quite long, so we have downloaded the results after each analysis. To continue from this point here on, upload the downloaded files back into your google colab session in a folder at root called `"analysis_results"`.

## 4. Export results

Now we can export the results as IIIF Manifests to be viewed in Millefeuille. We shall create a Manifest identical to the first, but add annotations for each shot detection.

In [ ]:
PATH_ON_SERVER = "manifests/plozevet/sources"

for analysis_result_path in os.listdir("analysis_results"):

  # Retrieve analysis results as a dict:
  with open(os.path.join("analysis_results", analysis_result_path), 'r') as f:
    analysis_results = json.load(f)

  # Load the original manifest to get the metadata:
  original_manifest = mfapi.read_manifest(analysis_results["manifest_path"])

  manifest = mfapi.Manifest(manifest_path = PATH_ON_SERVER, logo = "https://mediatheque.landerneau.bzh/medias/sites/3/2019/12/logo_CDB_1.jpg")
  manifest.label = {"en" : [original_manifest.label["en"][0] + f" (shot decomposition)"]}
  manifest.metadata = original_manifest.metadata
  manifest.requiredStatement = original_manifest.requiredStatement
  manifest.provider = original_manifest.provider
  manifest.add_canvas_from_media(analysis_results["media_path"])

  annotation_page = manifest.add_annotation_page()
  num_shots = len(analysis_results["shots"])

  for i, shot_data in enumerate(analysis_results["shots"]):
    annotation_page.add_annotation(
        start = shot_data[0],
        end = shot_data[1],
        label = f"Shot {i + 1}/{num_shots}"
    )

  files.download(manifest.to_json())

Finally, we can clear up.

In [ ]:
mfapi.clean_up()